<a href="https://colab.research.google.com/github/dream80/DeepFaceLab_Colab/blob/master/DeepFaceLab_Colab_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

无需多说，一路运行即可！

此脚本由**[托尼是塔克](https://www.tonyisstark.com)**创建，仅用于学习，请勿滥用。如有疑问，拉到最后！

QQ群：659480116  
教程地址：https://www.deepfaker.xyz

In [ ]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

Fri Nov 12 06:08:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 第一步 准备工作
 

In [ ]:
#@title 挂载云盘
from google.colab import drive
drive._mount('/content/drive')

In [ ]:
#@title 初始化项目

import os 
root = "/content/drive/Shareddrives/dl/" #@param {type:"string"}
tensorflow = "2" #@param [1,2]
%cd $root
!mkdir DeepFaceLab
deepfacelab=os.path.join(root,"DeepFaceLab")
workspace=os.path.join(deepfacelab,"workspace")
deepfacelab_cloab=os.path.join(deepfacelab,"DeepFaceLab_Colab")

if tensorflow=="1":
  %tensorflow_version 1.x
  !sudo apt-get install cuda-10-0
else:
  %tensorflow_version 2.x



/content/drive/Shareddrives/dl
mkdir: cannot create directory ‘DeepFaceLab’: File exists


In [ ]:
#@title 准备素材

online = True #@param {type:"boolean"}
upload = "data_src/aligned" #@param ["data_src/aligned","data_dst/aligned","model","data_src","data_dst"]

if online :
  %cd $deepfacelab
  !rm -rf workspace
  !git clone https://github.com/dream80/DFLWorkspace.git workspace
else:
  from google.colab import files
  import shutil
  upload_path = os.path.join(workspace,upload)
  %cd $upload_path
  #if os.path.isdir(upload_path):
  #shutil.rmtree(upload_path)
  uploaded = files.upload()
  '''for filename in uploaded.keys():
    print(basepath+"/"+filename)
    print(upload_path+"/"+filename)
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, filename))

  '''


# 第二部 安装软件

In [ ]:
#@title 开始安装

Version = "v3.07.20" #@param ["v2.02.03", "v2.02.23b", "v2.02.28","v2.03.07","v2.03.15","v2.06.01","v2.06.19","v2.08.02","v3.01.04","v3.07.20","last"]
%cd $deepfacelab
!rm -fr DeepFaceLab_Colab

if Version=="620":
  print("620版加载中....")
  # 获取DFL源代码v1.6.1稳定版
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd $deepfacelab_cloab
  !pip install -r requirements-colab.txt  
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/dream80/DeepFaceLab_Colab.git DeepFaceLab_Colab
  %cd $deepfacelab_cloab
  !pip install -r requirements-colab.txt
else:
  print(Version+" 加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"  
  !git $cmd
  %cd $deepfacelab_cloab
  !pip install -r requirements-colab.txt




/content/drive/Shareddrives/dl/DeepFaceLab
v3.07.20 加载中....
Cloning into 'DeepFaceLab_Colab'...
remote: Enumerating objects: 4151, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 4151 (delta 46), reused 90 (delta 40), pack-reused 4017
Receiving objects: 100% (4151/4151), 785.02 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (2601/2601), done.
Note: checking out '68e8d1ef928f39d1c3cfd14f82685e26a9ee9f6b'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

Checking out files: 100% (439/439), done.
/content/drive/Shareddrives/dl/DeepFaceLab/DeepFaceLab_Colab
     |█████████

# 第三步. 提取脸部

In [ ]:
#@title 开始提取SRC

%cd $deepfacelab_cloab
print("===分解视频==================================")
!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
print("===提取人脸==================================")
!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd
print("===打包人脸==================================")
!python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset


In [ ]:
#@title 开始提取DST
%cd $deepfacelab_cloab
print("===分解视频==================================")
!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
print("===提取人脸==================================")
!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --output-debug
print("===打包人脸==================================")
!python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset




```
# 此内容为代码格式
```

# 第四步. 训练模型


In [ ]:
#@title 训练模型
Model = "SAEHD" #@param ["SAEHD","AMP","Quick96","XSeg"]
%cd $deepfacelab_cloab
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

In [ ]:
#@title SRC-SRC
Model = "AMP" #@param ["SAEHD","AMP","Quick96","XSeg"]
%cd $deepfacelab_cloab
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_src/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

/content/drive/Shareddrives/dl/DeepFaceLab/DeepFaceLab_Colab
Running trainer.

Choose one of saved models, or enter a name to create a new model.
[r] : rename
[d] : delete

[0] : AMP - latest
 : 
0
Loading AMP_AMP model...


Choose one or several GPU idxs (separated by comma).

[CPU] : CPU
  [0] : Tesla V100-SXM2-16GB

[0] Which GPU indexes to choose? : 0

Initializing models: 100% 5/5 [00:18<00:00,  3.72s/it]
Loaded 27382 packed faces from /content/drive/Shareddrives/dl/DeepFaceLab/workspace/data_src/aligned
================= Model Summary =================
==                                             ==
==            Model name: AMP_AMP              ==
==                                             ==
==     Current iteration: 379290               ==
==                                             ==
==--------------- Model Options ---------------==
==                                             ==
==            resolution: 224                  ==
==             face_type: wf       

# 第五步. 转换输出



In [ ]:
#@title 应用模型

Model = "SAEHD" #@param ["SAEHD", "Quick96"]
%cd $deepfacelab_cloab
cmd = " main.py merge --output-mask-dir ../workspace/data_dst/merged_mask --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [ ]:
#@title 合成视频
%cd $deepfacelab_cloab

format = "mp4" #@param ["mp4","avi","mov"]
lossless = False #@param {type:"boolean"}

merged = "main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio"
merged_mask = "main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged_mask --output-file ../workspace/result_mask.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio --lossless"


if lossless :
  merged=merged+" --include-audio --lossless"

!python $merged
!python $merged_mask



#第六步. 继续训练



In [ ]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD" , "Quick96"]

#挂载
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


import os 
root = "/content/drive/Shareddrives/dl/" #@param {type:"string"}
tensorflow = "2.x" #@param ["1.x","2.x"]
%cd $root
!mkdir DeepFaceLab
deepfacelab=os.path.join(root,"DeepFaceLab")
workspace=os.path.join(deepfacelab,"workspace")
deepfacelab_cloab=os.path.join(deepfacelab,"DeepFaceLab_Colab")

if tensorflow=="1.x":
  %tensorflow_version 1.x
else:
  %tensorflow_version 2.x

%cd $deepfacelab_cloab
# 安装Python依赖
!pip install -r requirements-colab.txt
# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd 

Done.


# 工具



打包素材。可以加快上传，下载，加载的速度。

In [ ]:
#@title 开始排序
target = "src" #@param ["src","dst"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned"
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned "

!python $cmd

In [ ]:
#@title 素材打包

target = "dst" #@param ["src","dst"]
%cd $deepfacelab_cloab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset
 
  

In [ ]:
#@title 素材解包

target = "src" #@param ["src","dst"]
%cd $deepfacelab_cloab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset
 
  

In [ ]:
#@title 素材增强

target = "src" #@param ["src"]
%cd $deepfacelab_cloab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned
 
  

# 其他

  QQ群：659480116  
    
 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com
 
 使用教程：https://www.wxp123.me
 
 
不求别的，只求在github给个小星星^_^

https://github.com/dream80/DeepFaceLab_Colab

右上角star 谢谢!!!  
  
